In [1]:
!pip install transformers
!pip install accelerate>=0.26.0
!pip install datasets
!pip install -U bitsandbytes
!pip install sae_lens
!pip install huggingface_hub
import json
import os
import datasets
from sae_lens import SAE
import random
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from huggingface_hub import notebook_login
notebook_login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 57.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.4 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━

In [2]:
from transformer_lens import HookedTransformer

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
model = HookedTransformer.from_pretrained(model_name, device="cuda")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [3]:
from sae_lens import SAE

sae, cfg_dict, sparsity = SAE.from_pretrained(release="llama-3-8b-it-res-jh", sae_id="blocks.25.hook_resid_post", device="cuda")

cfg.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

sae_weights.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

sparsity.safetensors:   0%|          | 0.00/262k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/sae_lens/sae.py:644: UserWarning: norm_scaling_factor not found for llama-3-8b-it-res-jh and blocks.25.hook_resid_post, but normalize_activations is 'expected_average_only_in'. Skipping normalization folding.
  warnings.warn(


In [4]:
# get hook point
hook_point = sae.cfg.hook_name
print(hook_point)

blocks.25.hook_resid_post


In [32]:
from datasets import load_dataset
dataset = load_dataset("gsm8k", "main")

In [33]:
train_questions = [item["question"] for item in dataset["train"]]
test_questions = [item["question"] for item in dataset["test"]]

In [7]:
sv_prompt = " The Golden Gate Bridge"
sv_logits, cache = model.run_with_cache(sv_prompt)
tokens = model.to_tokens(sv_prompt)
print(tokens)

# get the feature activations from our SAE
sv_feature_acts = sae.encode(cache[hook_point])

# get sae_out
sae_out = sae.decode(sv_feature_acts)

# print out the top activations, focus on the indices
print(torch.topk(sv_feature_acts, 3))

tensor([[128000,    578,  18288,  30343,  20467]], device='cuda:0')
torch.return_types.topk(
values=tensor([[[12.1599, 12.1552, 12.0955],
         [ 9.5191,  5.9392,  3.5369],
         [17.2081,  2.9424,  2.5228],
         [ 5.6462,  4.2330,  3.9888],
         [ 6.4601,  2.9334,  2.7729]]], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([[[61272,  3179, 28003],
         [38702,   607, 63905],
         [44661, 21229, 37976],
         [ 2999, 23747,  5992],
         [   87, 62317, 23747]]], device='cuda:0'))


In [9]:
last_token_index = sv_feature_acts.shape[1] - 1  

last_token_activations = sv_feature_acts[0, last_token_index, :]

print(f"Shape of last token activations: {last_token_activations.shape}")  

print("Last token's all SAE feature activations:")
print(last_token_activations)

print(torch.topk(last_token_activations, 65536))
print(last_token_activations[:88])

Shape of last token activations: torch.Size([65536])
Last token's all SAE feature activations:
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0',
       grad_fn=<SliceBackward0>)
torch.return_types.topk(
values=tensor([6.4601, 2.9334, 2.7729,  ..., 0.0000, 0.0000, 0.0000], device='cuda:0',
       grad_fn=<TopkBackward0>),
indices=tensor([   87, 62317, 23747,  ..., 65533, 65534, 65535], device='cuda:0'))
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.00

In [34]:
import torch
import random
from sae_lens import SAE

SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)


# Generate N=10 pairs of prompts with and without role-play context
N = 10
role_play_prompt = "As a highly qualified mathematics teacher, you excel at solving problems systematically and explaining solutions with clarity. I am your student, eager to learn. Please solve the following problem."

selected_questions = [random.choice(train_questions) for _ in range(N)]

with_role_prompts = [
    f"{role_play_prompt} {q} Please output your final answer (a number) after 'Output: '"
    for q in selected_questions
]
without_role_prompts = [
    f"{q} Please output your final answer (a number) after 'Output: '"
    for q in selected_questions
]

# print(with_role_prompts)
# print(without_role_prompts)

# Initialize lists to store SAE activations
with_role_feature_acts_list = []
without_role_feature_acts_list = []

# Process each prompt and collect activations
for i in range(N):
    with_role_logits, with_role_cache = model.run_with_cache(with_role_prompts[i], prepend_bos=True)
    without_role_logits, without_role_cache = model.run_with_cache(without_role_prompts[i], prepend_bos=True)

    with_role_feature_acts = sae.encode(with_role_cache[hook_point])
    without_role_feature_acts = sae.encode(without_role_cache[hook_point])

    with_role_last_token_index = with_role_feature_acts.shape[1]-1
    without_role_last_token_index = without_role_feature_acts.shape[1]-1

    with_role_last_token_activations = with_role_feature_acts[0, with_role_last_token_index, :]
    without_role_last_token_activations = without_role_feature_acts[0, without_role_last_token_index, :]

    with_role_feature_acts_list.append(with_role_last_token_activations)
    without_role_feature_acts_list.append(without_role_last_token_activations)

print(with_role_feature_acts_list)
print(without_role_feature_acts_list)

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 79.15 GiB of which 23.25 MiB is free. Process 929212 has 79.12 GiB memory in use. Of the allocated memory 78.18 GiB is allocated by PyTorch, and 456.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
with_role_feature_acts = torch.stack(with_role_feature_acts_list)  # (N, feature_dim)
without_role_feature_acts = torch.stack(without_role_feature_acts_list)  # (N, feature_dim)

delta_h = ((with_role_feature_acts > 0).float() - (without_role_feature_acts > 0).float())

sensitivity_score = (delta_h > 0).float().mean(dim=0)  
k = 10
top_k_indices = torch.topk(sensitivity_score, k=k).indices
top_k_features = sensitivity_score[top_k_indices]


print(f"Top-{k} Role-Play Influenced Features (Indices & Sensitivity Scores):")
for idx, score in zip(top_k_indices.tolist(), top_k_features.tolist()):
    print(f"Feature {idx}: Sensitivity {score:.4f}")

Top-10 Role-Play Influenced Features (Indices & Sensitivity Scores):
Feature 44566: Sensitivity 1.0000
Feature 25959: Sensitivity 1.0000
Feature 24268: Sensitivity 0.9000
Feature 57884: Sensitivity 0.7000
Feature 20374: Sensitivity 0.6000
Feature 9300: Sensitivity 0.6000
Feature 61792: Sensitivity 0.5000
Feature 34989: Sensitivity 0.4000
Feature 53795: Sensitivity 0.4000
Feature 64089: Sensitivity 0.3000


In [11]:
top_k_activations = with_role_feature_acts[:, top_k_indices]  # (N, k)

mean_activation = top_k_activations.mean(dim=0)  # (k,)

std_activation = top_k_activations.std(dim=0)  # (k,)

β = 1.0  

steering_strengths = mean_activation + β * std_activation  # (k,)

steering_vectors = sae.W_dec[top_k_indices]  

steering_shift = (steering_strengths[:, None] * steering_vectors).sum(dim=0)  # (feature_dim,)


In [12]:

def steering_hook(resid_pre, hook):
    
    if resid_pre.shape[1] == 1:
        return

    position = resid_pre.shape[1] - 1  
    resid_pre[:, position, :] += steering_shift  


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None):
    
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            input=tokenized,
            max_new_tokens = 512,
        )
    return result


In [29]:
from tqdm import tqdm

def run_generate(example_prompt):
    
    model.reset_hooks()
    editing_hooks = [(f"blocks.25.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt], editing_hooks, seed=42
    )

    res_str = model.to_string(res[:, 1:])  
    return res_str[0]


In [31]:
for i in tqdm(range(len(test_questions)), desc="Generating Responses", unit="prompt"):
    example_prompt = test_questions[i] + " Please output your final answer (a number) after 'Output: ' at the end of your response." 
    response = run_generate(example_prompt)  
      
    print(f"Question {i+1}: {example_prompt}")  
    print(f"Response {i+1}: {response}")  


Generating Responses:   0%|          | 0/1319 [00:00<?, ?prompt/s]

  0%|          | 0/512 [00:00<?, ?it/s]

Generating Responses:   0%|          | 1/1319 [00:23<8:41:45, 23.75s/prompt]


Question 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Please output your final answer (a number) after 'Output: ' at the end of your response.
Response 1: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? Please output your final answer (a number) after 'Output: ' at the end of your response. Output: $26
Janet's ducks lay 16 eggs per day. So originally, she has 16 ducks eggs. Every morning, she eats 3, and bakes muffins with 4. So she eats/bakes 3 + 4 = 7. So everyday she is left with 16 - 7 = 9 fresh duck eggs. She sells t

  0%|          | 0/512 [00:00<?, ?it/s]

...

Generating Responses:   1%|          | 9/1319 [02:58<7:02:49, 19.37s/prompt]


Question 9: John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot something very important at home.  He tries to get home in 4 hours but spends the first 2 hours in standstill traffic.  He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time of the 4 hours going at 80 mph.  How far is he from home at the end of those 4 hours? Please output your final answer (a number) after 'Output: ' at the end of your response.
Response 9: John drives for 3 hours at a speed of 60 mph and then turns around because he realizes he forgot something very important at home.  He tries to get home in 4 hours but spends the first 2 hours in standstill traffic.  He spends the next half-hour driving at a speed of 30mph, before being able to drive the remaining time of the 4 hours going at 80 mph.  How far is he from home at the end of those 4 hours? Please output your final answer (a number) after 'Output: ' at the end o

  0%|          | 0/512 [00:00<?, ?it/s]

Generating Responses:   1%|          | 9/1319 [03:15<7:55:15, 21.77s/prompt]


KeyboardInterrupt: 